In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import os


PICTURES_FOLDER = "pictures"
os.makedirs(PICTURES_FOLDER, exist_ok=True)

SLEEP_STAGES_COLORS = {
    0: "blue",
    1: "green",
    2: "red",
    3: "black",
    4: "orange",
}

In [2]:
train_file = "kaggle_data/X_train.h5/X_train.h5"
test_file = "kaggle_data/X_test.h5/X_test.h5"

h5_train = h5py.File(train_file, mode='a')
h5_test = h5py.File(test_file, mode='a')

y_train = pd.read_csv("kaggle_data/y_train.csv", index_col=0, squeeze=True)

In [13]:
def update_globals():
    features = [feat for feat in h5_train.keys() if feat not in ('index', 'index_absolute', 'index_window')]
    frequencies = {feat: h5_train[feat][0].size / 30 for feat in features}
    return features, frequencies
    
FEATURES, FREQUENCIES = update_globals()
print("FEATURES =", FEATURES)
print("FREQUENCIES =", FREQUENCIES)

FEATURES = ['accel_norm', 'eeg_1', 'eeg_2', 'eeg_3', 'eeg_4', 'eeg_5', 'eeg_6', 'eeg_7', 'pulse', 'speed_norm', 'speed_x', 'speed_y', 'speed_z', 'x', 'y', 'z']
FREQUENCIES = {'accel_norm': 10.0, 'eeg_1': 50.0, 'eeg_2': 50.0, 'eeg_3': 50.0, 'eeg_4': 50.0, 'eeg_5': 50.0, 'eeg_6': 50.0, 'eeg_7': 50.0, 'pulse': 10.0, 'speed_norm': 10.0, 'speed_x': 10.0, 'speed_y': 10.0, 'speed_z': 10.0, 'x': 10.0, 'y': 10.0, 'z': 10.0}


In [4]:
# HELPERS

def make_timeline(freq):
    """
    ARGS:
        freq (int): frequency in Hertz
    
    RETURNS:
        (pd.timedelta_range) : timestamps for a signal sampled at <freq> Hz for 30 seconds
    """
    return pd.timedelta_range(start='0s', end='30s', periods=freq*30)


def make_full_timeline(windows, freq):
    # test there is no missing data
    deltas = np.unique(np.diff(windows))
    assert (len(deltas) == 1) and (int(deltas[0]) == 1)
    return pd.timedelta_range(start='0s',
                              end=pd.to_timedelta('30s') * (windows[-1] + 1),
                              periods=freq * 30 * (windows[-1] + 1))

def get_subject_ids(h5_file):
    return np.unique(h5_file["index"][:])

    
def get_subject_boundaries(h5_file, subject_id, ready_to_use=True):
    """
    Helper function to select data relating to a given subject (on numpy arrays)
    
    ARGS:
        h5_file (h5py.File)
        subject_id (int)
        ready_to_use (bool, default=True): return a slice or a tuple
        
    RETURNS:
        subject_boundaries : (slice) (index_start, index_end+1) if <ready_to_use>
                             (tuple) (index_start, index_end) if not <ready_to_use>
                        
    """
    sids = h5_file['index'][:]
    start = np.argmax(sids == subject_id)
    end = len(sids) - 1 - np.argmax(sids[::-1] == subject_id)
    
    indexers = h5_file['index_absolute'][:]
    start = indexers[start]
    end = indexers[end]
    if ready_to_use:
        return slice(start, end + 1) # for numpy arrays
    return (start, end)


def get_subject_feature_signals(h5_file, subject_id, feature, as_timeseries=False):
    """
    Get the full timeseries for a given (subject_id, feature) pair.
    
    ARGS:
        h5_file (h5py.File)
        subject_id (int)
        feature (str)
        
    RETURNS:
        timeseries : (pd.Series if <as_timeseries>) represents the <feature> timeseries of the subject 
                     (list[np.array[?]] if not <as_timeseries>) list of <feature> signals from the subject
    """
    # Fetch subject boundaries
    boundaries = get_subject_boundaries(h5_file, subject_id)
    # Retrieve samples
    feature_timeseries = h5_file[feature][boundaries]
    if not as_timeseries:
        return feature_timeseries
    feature_timeseries = np.concatenate(feature_timeseries, axis=0)
    # Build timeline
    feature_frequency = FREQUENCIES[feature]
    windows = h5_file['index_window'][boundaries]
    timeline = make_full_timeline(windows, feature_frequency)
    return pd.Series(data=feature_timeseries, index=timeline)


def get_subject_sleep_stage(subject_id):
    start, end = get_subject_boundaries(h5_train, subject_id, ready_to_use=False)
    return y_train.loc[start:end] # because loc includes <end> (different behaviour than numpy arrays)
    

def _create_speed_and_acceleration(h5_file, overwrite=False, verbose=True):
    """
    a[t] = (v[t] - v[t-1]) / dt 
    ===> v[t] = sum_{s=0}^{t} a[s] (+ v[-1] = 0)
    """
    freq = 10
    dt = 1 / freq
    
    # Create datasets if required
    if "accel_norm" in h5_file.keys() and not overwrite:
        return None
    shape, dtype = h5_file["x"].shape, h5_file["x"].dtype
    for name in ["accel_norm", "speed_x", "speed_y", "speed_z", "speed_norm"]:
        try:
            h5_file.create_dataset(name, shape=shape, dtype=dtype)
        except:
            pass
    
    # Initiate subject id
    sid = -1
    for ix in range(shape[0]):
        if sid != h5_file["index"][ix]:
            sid = h5_file["index"][ix]
            speed = np.array([[0, 0, 0]])
            if verbose:
                print(f"SUBJECT #{sid}")
        # acceleration
        accel = np.stack([h5_file[feat][ix] for feat in ("x", "y", "z")], axis=-1)
        h5_file["accel_norm"][ix] = np.linalg.norm(accel, ord=2, axis=1)
        # speed
        speed = speed + np.cumsum(accel, axis=0) * dt
        h5_file["speed_x"][ix] = speed[:, 0]
        h5_file["speed_y"][ix] = speed[:, 1]
        h5_file["speed_z"][ix] = speed[:, 2]
        h5_file["speed_norm"][ix] = np.linalg.norm(speed, ord=2, axis=1)
        # speed for next iteration
        speed = speed[[-1], :]
    return None
        
    

"""
# FOURIER

from scipy.fft import fft

def get_spectrum(seq, fs):
    ft_modulus = np.abs(fft(seq))
    # The signal is real so the spectrum is symmetric 
    if len(seq) % 2 == 0:
        ft_modulus = ft_modulus[:len(seq) // 2]
    else:
        ft_modulus = ft_modulus[:len(seq) // 2 + 1]
    freqs = np.arange(0, len(ft_modulus)) * fs / len(seq) # frequencies of the spectrum
    return pd.Series(data=ft_modulus, index=freqs)
    

def get_spectrum_maxima(seq, fs, thresh=0.1):
    spectrum = get_spectrum(seq, fs)
    delta_left = np.diff(spectrum, prepend=spectrum[0] - 1) > 0 # ascending
    delta_right = np.diff(spectrum[::-1], prepend=spectrum[-1] - 1)[::-1] > 0 # descending
    ix_keep = np.logical_and(delta_left, delta_right) # local maximum
    spectrum_util = spectrum.loc[ix_keep]
    spectrum_util = spectrum_util.loc[spectrum_util > spectrum_util.max() * thresh]
    return spectrum_util

# ACCELEROMETER

def acceleration_to_speed(accel_arr):
    if accel_arr.ndim == 2:
        return np.cumsum(accel_arr, axis=1)
    return np.cumsum(accel_arr)

def vec_to_norm(arr):
    return np.linalg.norm(arr, axis=1)
"""

# Create speed and acceleration
_create_speed_and_acceleration(h5_train, overwrite=False, verbose=True)
_create_speed_and_acceleration(h5_test, overwrite=False, verbose=True)
FEATURES, FREQUENCIES = update_globals()
print(FEATURES)

SUBJECT #0
SUBJECT #1
SUBJECT #2
SUBJECT #3
SUBJECT #4
SUBJECT #5
SUBJECT #6
SUBJECT #7
SUBJECT #8
SUBJECT #9
SUBJECT #10
SUBJECT #11
SUBJECT #12
SUBJECT #13
SUBJECT #14
SUBJECT #15
SUBJECT #16
SUBJECT #17
SUBJECT #18
SUBJECT #19
SUBJECT #20
SUBJECT #21
SUBJECT #22
SUBJECT #23
SUBJECT #24
SUBJECT #25
SUBJECT #26
SUBJECT #27
SUBJECT #28
SUBJECT #29
SUBJECT #30
SUBJECT #31
SUBJECT #32
SUBJECT #33
SUBJECT #34
SUBJECT #35
SUBJECT #36
SUBJECT #37
SUBJECT #38
SUBJECT #39
SUBJECT #40
SUBJECT #41
SUBJECT #42
SUBJECT #43
SUBJECT #44
SUBJECT #45
SUBJECT #46
SUBJECT #47
SUBJECT #48
SUBJECT #49
SUBJECT #50
SUBJECT #51
SUBJECT #52
SUBJECT #53
SUBJECT #54
SUBJECT #55
SUBJECT #56
SUBJECT #57
SUBJECT #58
SUBJECT #59
SUBJECT #60
['accel_norm', 'eeg_1', 'eeg_2', 'eeg_3', 'eeg_4', 'eeg_5', 'eeg_6', 'eeg_7', 'index', 'index_absolute', 'index_window', 'pulse', 'speed_norm', 'speed_x', 'speed_y', 'speed_z', 'x', 'y', 'z']


In [5]:
# Example
get_subject_feature_signals(h5_train, 1, "eeg_1", as_timeseries=False)

In [6]:
# Example
get_subject_feature_signals(h5_train, 1, "eeg_1", as_timeseries=True)

In [7]:
"""
def plot_subject_quantiles(subject_id, q_inf=0.025, q_sup=0.975, n_quantiles=20):
    sleep_states = get_subject_sleep_state(subject_id)
    qts = np.linspace(q_inf, q_sup, n_quantiles).round(3)
    for feature in FEATURES:
        signal = get_subject_feature_signals(h5_train, subject_id, feature, as_timeseries=False)
        size = signal[0].size
        signal_by_state = pd.Series(data=np.concatenate(signal, axis=0),
                                    index=np.repeat(sleep_states.values, size))
        qt_df = signal_by_state.groupby(signal_by_state.index).quantile(qts)
        qt_df.unstack(0).plot()
        # sns.heatmap(qt_df.unstack(0))
        plt.title(feature)
        plt.show()
    return None
"""

'\ndef plot_subject_quantiles(subject_id, q_inf=0.025, q_sup=0.975, n_quantiles=20):\n    sleep_states = get_subject_sleep_state(subject_id)\n    qts = np.linspace(q_inf, q_sup, n_quantiles).round(3)\n    for feature in FEATURES:\n        signal = get_subject_feature_signals(h5_train, subject_id, feature, as_timeseries=False)\n        size = signal[0].size\n        signal_by_state = pd.Series(data=np.concatenate(signal, axis=0),\n                                    index=np.repeat(sleep_states.values, size))\n        qt_df = signal_by_state.groupby(signal_by_state.index).quantile(qts)\n        qt_df.unstack(0).plot()\n        # sns.heatmap(qt_df.unstack(0))\n        plt.title(feature)\n        plt.show()\n    return None\n'

In [16]:
def robust_rescale(df):
    """
    X_rescaled = (X - MED(X)) / MED(|X - MED(X)|)
    """
    med = df.median()
    med_spread = (df - df.median()).abs().median()
    # df_rescaled = (df - med) / med_spread
    return (df - med) / med_spread

def min_max_rescale(df):
    min_ = df.min()
    max_ = df.max()
    return (df - min_) / (max_ - min_)
    
def z_rescale(df): 
    mean = df.mean()
    std = df.std()
    return (df - mean) / std

def save_feature_quantiles(feature,
                           inf_qt=0.025,
                           sup_qt=0.975,
                           n_quantiles=21,
                           robust_rescaling=False,
                           overwrite=False,
                           verbose=True):
    """
    See pictures/quantile_plots
    
    Can be improved (make robust and not robust qplots simultaneously)
    """
    # Make directory if it does not exist
    qplot_dir = os.path.join(PICTURES_FOLDER, f"quantile_plots")
    os.makedirs(qplot_dir, exist_ok=True)
    # Escape if not overwrite and already done
    qplot_fname = os.path.join(qplot_dir, f'{feature}{"--rescaled" if robust_rescaling else ""}.png')
    if (not overwrite) and os.path.exists(qplot_fname):
        return None
    # Otherwise,
    subject_ids = get_subject_ids(h5_train)
    quantiles = np.linspace(inf_qt, sup_qt, n_quantiles).round(3)
    subjects_quantiles = dict()
    for cnt, sid in enumerate(subject_ids):
        if verbose:
            print(f"--> SUBJECT {cnt+1}/{len(subject_ids)} (RESCALE = {str(robust_rescale)})")
        # Robust representation of the signal
        signal = get_subject_feature_signals(h5_train, sid, feature, as_timeseries=False)
        size = signal[0].size
        signal = pd.Series(np.concatenate(signal))
        if robust_rescaling:
            signal = robust_rescale(signal)
        # Behaviour by sleep stage
        sleep_stages = get_subject_sleep_stage(sid).values
        signal_by_stage = signal.groupby(np.repeat(sleep_stages, size))
        subjects_quantiles[sid] = signal_by_stage.quantile(quantiles).unstack(0)
        
    fig, axes = plt.subplots(10, 3, figsize=(10, 40))
    for ax, sid in zip(np.ravel(axes), subject_ids):
        subjects_quantiles[sid].plot(ax=ax, title=f"Subject {sid}", color=SLEEP_STAGES_COLORS)
    plt.savefig(qplot_fname)
    plt.clf()
    return subjects_quantiles


# TO WRITE QUANTILE PLOTS IN pictures/quantile_plots
for i, feat in enumerate(FEATURES):
    print(f"========= FEATURE {i+1}/{len(FEATURES)} =========")
    save_feature_quantiles(feat, robust_rescaling=False, overwrite=False, verbose=True)
    save_feature_quantiles(feat, robust_rescaling=True, overwrite=False, verbose=True)


========= FEATURE 1/16 =========
========= FEATURE 2/16 =========
========= FEATURE 3/16 =========
========= FEATURE 4/16 =========
========= FEATURE 5/16 =========
========= FEATURE 6/16 =========
========= FEATURE 7/16 =========
========= FEATURE 8/16 =========
========= FEATURE 9/16 =========
========= FEATURE 10/16 =========
========= FEATURE 11/16 =========
========= FEATURE 12/16 =========
========= FEATURE 13/16 =========
========= FEATURE 14/16 =========
========= FEATURE 15/16 =========
========= FEATURE 16/16 =========


In [17]:
'''
def do_nothing(x):
    return x

def aggregate_stat(stat_func, char, dataset, y_vals, chunksize=1000):
    """stat_func must have axis kwarg and take 2d arrays as arg"""
    chunks_ix = np.array_split(y_vals.index, len(y_vals) / chunksize)
    final = pd.Series([list() for _ in np.unique(y_vals)], index=np.unique(y_vals))
    for cnt, ix in enumerate(chunks_ix): 
        # print(cnt, '/', len(chunks_ix))
        tmp = pd.Series(stat_func(dataset[char][ix.tolist()], axis=1), index=y_vals.loc[ix])
        tmp = tmp.groupby(tmp.index).agg(list)
        final = final + tmp.reindex(final.index, fill_value=list())
    return final

def custom_group_mean(char, dataset, y_vals, chunksize=1000):
    chunks_ix = np.array_split(y_vals.index, len(y_vals) / chunksize)
    for cnt, ix in enumerate(chunks_ix): 
        # print(cnt, '/', len(chunks_ix))
        mean_tmp = pd.Series(np.mean(dataset[char][ix.tolist()], axis=1), index=y_vals.loc[ix])
        mean_tmp = mean_tmp.groupby(mean_tmp.index).agg(['mean', 'size'])
        if cnt == 0:
            mean_df = mean_tmp
            continue
        mean_df.loc[:, 'mean'] = (mean_df.prod(axis=1) + mean_tmp.prod(axis=1)) / (mean_df['size'] + mean_tmp['size'])
        mean_df.loc[:, 'size'] = mean_df['size'] + mean_df['size']
    return mean_df['mean']

# custom_group_mean('eeg_1', h5_train, y_train)
averages = pd.concat(map(lambda x: custom_group_mean(x, h5_train, y_train, 1000), FEATURES), axis=1, keys=FEATURES)
'''

'\ndef do_nothing(x):\n    return x\n\ndef aggregate_stat(stat_func, char, dataset, y_vals, chunksize=1000):\n    """stat_func must have axis kwarg and take 2d arrays as arg"""\n    chunks_ix = np.array_split(y_vals.index, len(y_vals) / chunksize)\n    final = pd.Series([list() for _ in np.unique(y_vals)], index=np.unique(y_vals))\n    for cnt, ix in enumerate(chunks_ix): \n        # print(cnt, \'/\', len(chunks_ix))\n        tmp = pd.Series(stat_func(dataset[char][ix.tolist()], axis=1), index=y_vals.loc[ix])\n        tmp = tmp.groupby(tmp.index).agg(list)\n        final = final + tmp.reindex(final.index, fill_value=list())\n    return final\n\ndef custom_group_mean(char, dataset, y_vals, chunksize=1000):\n    chunks_ix = np.array_split(y_vals.index, len(y_vals) / chunksize)\n    for cnt, ix in enumerate(chunks_ix): \n        # print(cnt, \'/\', len(chunks_ix))\n        mean_tmp = pd.Series(np.mean(dataset[char][ix.tolist()], axis=1), index=y_vals.loc[ix])\n        mean_tmp = mean_tmp.

In [18]:
h5_train.close()
h5_test.close()